In [1]:
import pandas as pd
import torch
from peft import LoraConfig, TaskType, get_peft_model
from torch.utils.data import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    Trainer,
    TrainingArguments,
)

/home/tdutton/mambaforge/envs/dsl-final-project/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def load_model(
    model_name: str = "meta-llama/Meta-Llama-3-8B-Instruct", load_on_gpu: bool = True
) -> tuple[AutoModelForCausalLM, AutoTokenizer]:
    if load_on_gpu:
        config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=False,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16,
        )

        model = AutoModelForCausalLM.from_pretrained(
            model_name, low_cpu_mem_usage=True, quantization_config=config
        )
    else:
        model = AutoModelForCausalLM.from_pretrained(model_name, low_cpu_mem_usage=True)

    tokenizer = AutoTokenizer.from_pretrained(model_name)

    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        tokenizer.pad_token_id = tokenizer.eos_token_id

    return model, tokenizer

In [3]:
def tokenize_queries(tokenizer: AutoTokenizer, query: str, max_length: int = 1024) -> torch.Tensor:
    query = [
        {"role": "system", "content": "You are a recipe generator. Give the necessary ingredients and steps to create the input dish."},
        {"role": "user", "content": query},
    ]

    input_ids = tokenizer.apply_chat_template(
        query,
        padding='max_length', 
        max_length=max_length,
        add_generation_prompt=True,
        return_tensors="pt"
    )

    return input_ids

def tokenize_responses(tokenizer: AutoTokenizer, response: str, max_length: int = 1024) -> torch.Tensor:
    response = [
        {"role": "assistant", "content": response},
    ]

    labels = tokenizer.apply_chat_template(
        response,
        padding='max_length', 
        max_length=max_length,
        add_generation_prompt=True,
        return_tensors="pt"
    )

    return labels

In [4]:
class RecipeDataset(Dataset):
    def __init__(self, data: pd.DataFrame):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return {
            "input_ids": self.data.iloc[idx]["input_ids"][0],
            "labels": self.data.iloc[idx]["labels"][0],
        }

In [5]:
# Load the model & tokenizer
model, tokenizer = load_model()

# Add Lora Matrices
lora_config = LoraConfig(
    r=8,
    lora_alpha=8,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.2,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

peft_model = get_peft_model(model, lora_config)

Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.43s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
# Print th number of trainable parameters
print(f"Number of trainable parameters: {sum(p.numel() for p in peft_model.parameters() if p.requires_grad)}")

Number of trainable parameters: 3407872


In [7]:
# Load in the data
train_data = pd.read_csv("./data/train.csv")
test_data = pd.read_csv("./data/test.csv")

# Take the first 100 samples for training
train_data = train_data.head(100)

# Take the first 10 samples for evaluation
test_data = test_data.head(10)

In [8]:
# Tokenize the data
train_data["input_ids"] = train_data.apply(lambda row: tokenize_queries(tokenizer, row["query"]), axis=1)
train_data["labels"] = train_data.apply(lambda row: tokenize_responses(tokenizer, row["response"]), axis=1)

test_data["input_ids"] = test_data.apply(lambda row: tokenize_queries(tokenizer, row["query"]), axis=1)
test_data["labels"] = test_data.apply(lambda row: tokenize_responses(tokenizer, row["response"]), axis=1)

# Drop unecessary columns
train_data = train_data.drop(columns="query").drop(columns="response")
test_data = test_data.drop(columns="query").drop(columns="response")

In [9]:
# Setup trainer
training_args = TrainingArguments(
    output_dir="./checkpoints",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    learning_rate=1e-3,
    num_train_epochs=10,
    logging_steps=1,
    save_steps=10,
    log_level="info",
    report_to="wandb"
)

trainer = Trainer(model=peft_model, 
                      args=training_args,
                      train_dataset=RecipeDataset(train_data),
                      eval_dataset=RecipeDataset(test_data)
        )

/home/tdutton/mambaforge/envs/dsl-final-project/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [10]:
# Now train
trainer.train()

***** Running training *****
  Num examples = 100
  Num Epochs = 10
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 1,000
  Number of trainable parameters = 3,407,872
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: troydutton (troydutton-dev). Use `wandb login --relogin` to force relogin


Step,Training Loss
1,39.576000
2,26.439400
3,18.928500
4,11.289200
5,5.071600
6,3.464500
7,1.713400
8,1.028200
9,4.651400
10,2.265200


Saving model checkpoint to ./checkpoints/checkpoint-10
loading configuration file config.json from cache at /home/tdutton/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3-8B-Instruct/snapshots/e5e23bbe8e749ef0efcf16cad411a7d23bd23298/config.json
Model config LlamaConfig {
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128001,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 500000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.39.3",
  "use_cache": true,
  "vocab_size": 128256
}

Saving model checkpoint to ./checkpoints/checkpoint-20
loading configura